In [13]:
import simpy
import random
import numpy as np

def client(env, name, servers, dist_servers):
    # Process of client across the n servers
    print(f'{name} entered in the system at t={env.now:.2f}')
    
    for i in range(len(servers)):
        with servers[i].request() as req:
            yield req # wait for the server to accept me
            print(f'{name} started service in server {i+1} at t={env.now:.2f}')
            yield env.timeout(dist_servers[i]()) # wait until the server finishs with me
            print(f'{name} finished service in server {i+1} at t={env.now:.2f}')

    # Finished with all n servers
    print(f'{name} left the system at t={env.now:.2f}')

def setup(env, num_servers, dist_arrival, dist_servers):
    # Setups the simulation and creates the client´s process
    servers = [simpy.Resource(env) for _ in range(num_servers)]
    
    # Create clients
    i = 1
    while True:
        yield env.timeout(dist_arrival()) # wait until new client arrives
        env.process(client(env, f'Client {i}', servers, dist_servers)) # process new client
        i += 1

# Create environment
env = simpy.Environment()

# Set number of servers, and time distributions to use in each one
dist_servers = [ lambda: random.uniform(2, 30)
                ,lambda: np.random.geometric(1.0 / 10)
                ,lambda: max(0, random.gauss(12, 5))
                ,lambda: random.expovariate(1.0 / 50)
                ]

# Setup the environment, and set the time distribution for client arrival
env.process(setup(env, len(dist_servers), lambda: random.expovariate(1.0 / 5), dist_servers))
env.run(until=5000)  # Run the simulation for a period of time 


Client 1 entered in the system at t=16.65
Client 1 started service in server 1 at t=16.65
Client 2 entered in the system at t=27.50
Client 3 entered in the system at t=36.54
Client 4 entered in the system at t=36.79
Client 5 entered in the system at t=37.48
Client 1 finished service in server 1 at t=38.89
Client 1 started service in server 2 at t=38.89
Client 2 started service in server 1 at t=38.89
Client 6 entered in the system at t=39.14
Client 7 entered in the system at t=39.39
Client 8 entered in the system at t=47.23
Client 9 entered in the system at t=49.53
Client 10 entered in the system at t=53.61
Client 1 finished service in server 2 at t=54.89
Client 1 started service in server 3 at t=54.89
Client 1 finished service in server 3 at t=54.89
Client 1 started service in server 4 at t=54.89
Client 2 finished service in server 1 at t=55.40
Client 2 started service in server 2 at t=55.40
Client 3 started service in server 1 at t=55.40
Client 2 finished service in server 2 at t=56.4